In [50]:
img=Image.open("data/test/paper/color3.jpg").convert("L")
img.save("data/test/paper/color3.jpg")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg = Discriminator().to(device)
summary(vgg, (3, 256, 256))

In [37]:
from core.Generators import Generator,Generator_Mod,Generator2
from core.Discriminator import Discriminator,Discriminator_Mod
from Gan_train import Gray_GanTrainer
from config import CycleGANConfig as config
from core.data_loader import DefaultDataset,get_test_loader,get_gray_test_loader
from torchsummary import summary
import torch
import torchvision
import argparse
import torchvision.utils as tvutils
import os
import torch.nn as nn
from torchvision import transforms
from Gan_main import main
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils import data

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),
                                transforms.Resize([256,256])
                              ])
dataset=DefaultDataset('./data/test',transform=transform)
loader=data.DataLoader(dataset=dataset,batch_size=8)
torch_to_image = transforms.Compose([
    transforms.Normalize(mean=(-1, -1, -1), std=(2, 2, 2)),  # [-1, 1] to [0, 1]
    transforms.ToPILImage()
])
for img in loader:
  plt.axis('off')
  plt.imshow(torchvision.utils.make_grid(img,nrow=4,padding=5,pad_value=0.0,normalize=True).permute(1,2,0))
  plt.savefig("test",dpi=500)
  plt.show()

In [43]:
# 생성된 이미지 원하는데에 저장 for fid
torch_to_image = transforms.Compose([
    transforms.Normalize(mean=(-1, -1, -1), std=(2, 2, 2)),  # [-1, 1] to [0, 1]
    transforms.ToPILImage()
])
device=config.device
print("PyTorch running with device {0}".format(device))
G = Generator_Mod().to(device)

checkpoint=torch.load("checkpoints/CycleGAN/Mod/cartoonization/checkpoint-epoch-100.ckpt")
G.load_state_dict(checkpoint['G_state_dict'])

test_images=get_test_loader(root="data/l", batch_size=2, shuffle=False)
ix=0
G.eval()

for img,label in test_images:
    img=img.to(device)
    img=G(img).detach().cpu()
    for i in range(len(img)):
        image=torch_to_image(img[i])
        image.save(os.path.join("data/test/for_Mod_Cartoonizing",'{0}.jpg'.format(ix)))
        ix+=1


PyTorch running with device cuda
Preparing DataLoader for the generation phase...


In [36]:
# 생성된 이미지 원하는데에 저장 for fid
torch_to_image = transforms.Compose([
    transforms.Normalize(mean=(-1, -1, -1), std=(2, 2, 2)),  # [-1, 1] to [0, 1]
    transforms.ToPILImage()
])
device=config.device
print("PyTorch running with device {0}".format(device))
G = Generator_Mod().to(device)

checkpoint=torch.load("checkpoints/CycleGAN/Mod/cartoonization/checkpoint-epoch-100.ckpt")
G.load_state_dict(checkpoint['G_state_dict'])

test_images=get_test_loader(root="data/test/for_add_experiment/data/cartoon", batch_size=3, shuffle=False)
ix=0
G.eval()

for img in test_images:
    img=img.to(device)
    mod_img=transforms.Resize(64)(img)
    mod_img=nn.ZeroPad2d(96)(mod_img)
    img=G(img).detach().cpu()
    mod_generated_img=G(mod_img).detach().cpu()
    mod_img.detach().cpu()
    for image,mod_image in zip(img,mod_generated_img):
        mod_image=torch_to_image(mod_image)
        mod_image.save(os.path.join("data/test/for_add_experiment/transformed_data/cartoon/center_{0}.jpg".format(ix)))
        image=torch_to_image(image)
        image.save(os.path.join("data/test/for_add_experiment/result/cartoon/center_{0}.jpg".format(ix)))
        ix+=1


PyTorch running with device cuda
Preparing DataLoader for the generation phase...
